In [ ]:
# ==========================================
# WLASL LSTM TRAINING - Real-time Sign Language Recognition
# ==========================================
# Train MediaPipe + LSTM model on WLASL isolated signs for continuous real-time detection

import os
import numpy as np
import pandas as pd
import json
import glob
from tqdm import tqdm

print("✅ Imports loaded successfully!")

In [ ]:
# ==========================================
# 1. HELPER FUNCTIONS (Run this first!)
# ==========================================

import tensorflow as tf
from scipy import interpolate

# Target sequence length for LSTM
TARGET_FRAMES = 30

# Landmark counts (MediaPipe Holistic)
N_FACE = 468
N_POSE = 33
N_LEFT_HAND = 21
N_RIGHT_HAND = 21
TOTAL_LANDMARKS = N_FACE + N_POSE + N_LEFT_HAND + N_RIGHT_HAND  # 543

def load_parquet_landmarks(file_path):
    """Load landmarks from parquet file."""
    try:
        df = pd.read_parquet(file_path)
        return df
    except:
        return None

def parquet_to_array(df):
    """Convert parquet DataFrame to (T, 543, 3) array."""
    if df is None or len(df) == 0:
        return None
    
    n_frames = df['frame'].max() + 1
    
    # Initialize array with zeros
    data = np.zeros((n_frames, TOTAL_LANDMARKS, 3), dtype=np.float32)
    
    # Fill in landmarks
    for _, row in df.iterrows():
        frame_idx = int(row['frame'])
        lm_idx = int(row['landmark_index'])
        lm_type = row['type']
        
        # Map type to global index
        if lm_type == 'face':
            global_idx = lm_idx
        elif lm_type == 'left_hand':
            global_idx = N_FACE + lm_idx
        elif lm_type == 'pose':
            global_idx = N_FACE + N_LEFT_HAND + lm_idx
        elif lm_type == 'right_hand':
            global_idx = N_FACE + N_LEFT_HAND + N_POSE + lm_idx
        else:
            continue
        
        if global_idx < TOTAL_LANDMARKS and frame_idx < n_frames:
            data[frame_idx, global_idx, 0] = row['x']
            data[frame_idx, global_idx, 1] = row['y']
            data[frame_idx, global_idx, 2] = row['z']
    
    return data

def resample_sequence(data, target_len=TARGET_FRAMES):
    """Resample sequence to target length using linear interpolation."""
    current_len = data.shape[0]
    
    if current_len == target_len:
        return data
    
    # Create interpolation indices
    old_indices = np.linspace(0, current_len - 1, current_len)
    new_indices = np.linspace(0, current_len - 1, target_len)
    
    # Interpolate each landmark and coordinate
    resampled = np.zeros((target_len, TOTAL_LANDMARKS, 3), dtype=np.float32)
    
    for lm in range(TOTAL_LANDMARKS):
        for coord in range(3):
            f = interpolate.interp1d(old_indices, data[:, lm, coord], kind='linear')
            resampled[:, lm, coord] = f(new_indices)
    
    return resampled

print("✅ Helper functions defined!")
print(f"   Target frames: {TARGET_FRAMES}")
print(f"   Total landmarks: {TOTAL_LANDMARKS}")

In [ ]:
# ==========================================
# 1. EXPLORE WLASL LANDMARKS DATA
# ==========================================

# Path to WLASL landmarks
WLASL_DIR = "/kaggle/input/wlasl2000-landmarks/wlasl_landmarks"

# List all parquet files
parquet_files = glob.glob(f"{WLASL_DIR}/*.parquet")
print(f"📊 Total parquet files: {len(parquet_files)}")

# Load one sample to inspect structure
sample_file = "/kaggle/input/wlasl2000-landmarks/wlasl_landmarks/00336.parquet"
print(f"\n🔍 Inspecting sample file: {sample_file}")

df_sample = pd.read_parquet(sample_file)
print(f"\n📋 DataFrame shape: {df_sample.shape}")
print(f"📋 Columns: {list(df_sample.columns)}")
print(f"\n📋 First 10 rows:")
print(df_sample.head(10))

print(f"\n📋 Data types:")
print(df_sample.dtypes)

print(f"\n📋 Unique values:")
if 'type' in df_sample.columns:
    print(f"   Types: {df_sample['type'].unique()}")
if 'frame' in df_sample.columns:
    print(f"   Frames: {df_sample['frame'].min()} to {df_sample['frame'].max()}")
    print(f"   Total frames: {df_sample['frame'].nunique()}")

In [ ]:
# ==========================================
# 2. LOAD WLASL-100 WORD MAPPINGS
# ==========================================

# Load the correct label mapping (100 words from WLASL)
LABEL_MAPPING_PATH = "/kaggle/input/wlasl200-mapping/label_mapping.json"

print(f"📖 Loading WLASL-100 label mapping: {LABEL_MAPPING_PATH}")
with open(LABEL_MAPPING_PATH, 'r') as f:
    label_data = json.load(f)

gloss_to_label = label_data['gloss_to_label']
label_to_gloss = label_data['label_to_gloss']
num_classes = label_data['num_classes']

print(f"\n✅ Loaded label mapping!")
print(f"📊 Number of classes: {num_classes}")
print(f"📊 Total words: {len(gloss_to_label)}")

print(f"\n📝 First 20 words:")
for i, word in enumerate(list(gloss_to_label.keys())[:20], 1):
    label = gloss_to_label[word]
    print(f"   {i:2d}. {word:20s} → Label {label}")

print(f"\n📝 Last 10 words:")
for i, word in enumerate(list(gloss_to_label.keys())[-10:], 91):
    label = gloss_to_label[word]
    print(f"   {i:2d}. {word:20s} → Label {label}")

# 🎯 Strategy: Train LSTM on 100 Words

## ✅ What We Have:
- **100 words** from WLASL (accident, africa, all, apple, ...)
- **~11,980 parquet files** with landmarks
- **64 frames per video** (need to resample to 30)
- **Landmarks format:** frame, type, landmark_index, x, y, z

## 🎯 Next Steps:
1. **Load all videos** for our 100 words
2. **Resample** from 64 frames → 30 frames
3. **Build LSTM model** (3 LSTM layers + Dense)
4. **Train** (20-40 mins on GPU)
5. **Export weights** for local webcam inference

**Run cells 0-3 first to verify the data, then continue!** 🚀

In [ ]:
# ==========================================
# 5. CREATE VIDEO-TO-WORD MAPPING (CORRECT VERSION)
# ==========================================

print("📖 Loading WLASL_v0.3.json (full dataset)...")

WLASL_FULL = "/kaggle/input/wlasl-processed/WLASL_v0.3.json"

with open(WLASL_FULL, 'r') as f:
    wlasl_full = json.load(f)

print(f"✅ Loaded WLASL_v0.3.json")
print(f"   Type: {type(wlasl_full)}")

# Check structure
if isinstance(wlasl_full, list):
    print(f"   Total entries: {len(wlasl_full)}")
    print(f"\n🔍 Sample entry:")
    print(json.dumps(wlasl_full[0], indent=2)[:500])
    
    # Create mapping: video_id → word
    print(f"\n🔗 Creating video_id → word mapping...")
    
    video_to_word = {}
    word_to_videos = {}
    
    for entry in tqdm(wlasl_full, desc="Processing WLASL entries"):
        word = entry.get('gloss', None)
        
        # Only process words in our 100-word list
        if word and word in gloss_to_label:
            instances = entry.get('instances', [])
            
            for instance in instances:
                video_id = instance.get('video_id', None)
                if video_id:
                    video_to_word[video_id] = word
                    if word not in word_to_videos:
                        word_to_videos[word] = []
                    word_to_videos[word].append(video_id)
    
    print(f"\n✅ Mapping created!")
    print(f"   Total video instances: {len(video_to_word)}")
    print(f"   Words found: {len(word_to_videos)}/{num_classes}")
    
    # Show distribution
    print(f"\n? Videos per word (top 20):")
    sorted_words = sorted(word_to_videos.items(), key=lambda x: len(x[1]), reverse=True)
    for i, (word, videos) in enumerate(sorted_words[:20], 1):
        label = gloss_to_label[word]
        print(f"   {i:2d}. {word:20s} (Label {label:2d}) → {len(videos):3d} videos")
    
    # Check coverage
    missing_words = set(gloss_to_label.keys()) - set(word_to_videos.keys())
    if missing_words:
        print(f"\n⚠️ Missing {len(missing_words)} words:")
        print(f"   {sorted(list(missing_words))[:20]}")
    else:
        print(f"\n✅ All 100 words have video data!")
    
    # Show stats
    video_counts = [len(videos) for videos in word_to_videos.values()]
    print(f"\n📊 Video distribution:")
    print(f"   Min videos per word: {min(video_counts)}")
    print(f"   Max videos per word: {max(video_counts)}")
    print(f"   Mean videos per word: {np.mean(video_counts):.1f}")
    print(f"   Median videos per word: {np.median(video_counts):.1f}")

print(f"\n" + "="*60)
print(f"✅ MAPPING COMPLETE! Ready to load dataset.")
print(f"="*60)

In [ ]:
# ==========================================
# 6. LOAD AND PREPARE 100-WORD DATASET
# ==========================================
# Note: This creates the specific dataset for our target 100 words

print("📦 Loading dataset for 100 words...")

X_data = []  # Landmarks
y_data = []  # Labels
video_ids_list = []  # Track which video

# Load data for each word
for word, videos in tqdm(word_to_videos.items(), desc="Loading words"):
    label = gloss_to_label[word]
    
    for video_id in videos:
        # Construct file path
        file_path = f"{WLASL_DIR}/{video_id}.parquet"
        
        if not os.path.exists(file_path):
            continue
        
        # Load landmarks
        df = load_parquet_landmarks(file_path)
        if df is None:
            continue
        
        # Convert to array
        landmarks = parquet_to_array(df)
        if landmarks is None:
            continue
        
        # Resample to 30 frames
        landmarks_resampled = resample_sequence(landmarks, TARGET_FRAMES)
        
        # Flatten to (30, 543*3) = (30, 1629)
        landmarks_flat = landmarks_resampled.reshape(TARGET_FRAMES, -1)
        
        X_data.append(landmarks_flat)
        y_data.append(label)
        video_ids_list.append(video_id)

# Convert to numpy arrays
X_data = np.array(X_data, dtype=np.float32)
y_data = np.array(y_data, dtype=np.int32)

print(f"\n✅ Dataset loaded!")
print(f"   Total samples: {len(X_data)}")
print(f"   X shape: {X_data.shape}")
print(f"   y shape: {y_data.shape}")
print(f"   Unique classes: {len(np.unique(y_data))}")

# Check distribution
unique, counts = np.unique(y_data, return_counts=True)
print(f"\n📊 Samples per class:")
print(f"   Min: {counts.min()}")
print(f"   Max: {counts.max()}")
print(f"   Mean: {counts.mean():.1f}")

# 🎯 NEW STRATEGY: Pretrain + Fine-tune

## Phase 1: Pretrain on ALL 2,000 Words
- Use ALL WLASL data (~40,000 samples)
- Train for 50 epochs
- Save pretrained weights

## Phase 2: Fine-tune on 100 Words
- Load pretrained weights
- Train only on your 100 words
- Much better accuracy!

## Why This is Better:
✅ More data = better feature learning
✅ Transfer learning = higher accuracy  
✅ Better generalization
✅ Still fast on GPU

In [ ]:
# ==========================================
# 8. LOAD ALL 2,000 WORDS (PRETRAINING)
# ==========================================

print("📦 Loading ALL WLASL data for pretraining...")

# Create mapping for ALL words (not just 100)
all_word_to_videos = {}
all_video_to_word = {}

print("🔗 Creating mapping for ALL 2,000 words...")
for entry in tqdm(wlasl_full, desc="Processing all WLASL entries"):
    word = entry.get('gloss', None)
    
    if word:  # Include ALL words
        instances = entry.get('instances', [])
        
        for instance in instances:
            video_id = instance.get('video_id', None)
            if video_id:
                all_video_to_word[video_id] = word
                if word not in all_word_to_videos:
                    all_word_to_videos[word] = []
                all_word_to_videos[word].append(video_id)

print(f"\n✅ Full mapping created!")
print(f"   Total words: {len(all_word_to_videos)}")
print(f"   Total videos: {len(all_video_to_word)}")

# Create label mapping for all words
all_gloss_to_label = {word: i for i, word in enumerate(all_word_to_videos.keys())}
all_label_to_gloss = {i: word for word, i in all_gloss_to_label.items()}

print(f"\n📊 Full dataset stats:")
video_counts = [len(videos) for videos in all_word_to_videos.values()]
print(f"   Min videos per word: {min(video_counts)}")
print(f"   Max videos per word: {max(video_counts)}")
print(f"   Mean videos per word: {np.mean(video_counts):.1f}")
print(f"   Total samples: {sum(video_counts)}")

In [ ]:
# ==========================================
# 9. LOAD FULL DATASET (PRETRAINING)
# ==========================================

print("📦 Loading full dataset for pretraining...")
print("⚠️ This will take 10-20 minutes (loading ~40,000 samples)...")

X_full = []  # Landmarks
y_full = []  # Labels

# Load subset for faster pretraining (you can adjust)
MAX_SAMPLES_PER_WORD = 50  # Limit to 50 samples per word for faster training
loaded_samples = 0

for word, videos in tqdm(all_word_to_videos.items(), desc="Loading all words"):
    label = all_gloss_to_label[word]
    
    # Limit samples per word
    videos_to_load = videos[:MAX_SAMPLES_PER_WORD]
    
    for video_id in videos_to_load:
        file_path = f"{WLASL_DIR}/{video_id}.parquet"
        
        if not os.path.exists(file_path):
            continue
        
        # Load landmarks
        df = load_parquet_landmarks(file_path)
        if df is None:
            continue
        
        # Convert to array
        landmarks = parquet_to_array(df)
        if landmarks is None:
            continue
        
        # Resample to 30 frames
        landmarks_resampled = resample_sequence(landmarks, TARGET_FRAMES)
        landmarks_flat = landmarks_resampled.reshape(TARGET_FRAMES, -1)
        
        X_full.append(landmarks_flat)
        y_full.append(label)
        loaded_samples += 1

# Convert to numpy arrays
X_full = np.array(X_full, dtype=np.float32)
y_full = np.array(y_full, dtype=np.int32)

print(f"\n✅ Full dataset loaded!")
print(f"   Total samples: {len(X_full)}")
print(f"   X shape: {X_full.shape}")
print(f"   y shape: {y_full.shape}")
print(f"   Unique classes: {len(np.unique(y_full))}")

# Split for pretraining
from sklearn.model_selection import train_test_split
X_train_full, X_val_full, y_train_full, y_val_full = train_test_split(
    X_full, y_full, test_size=0.2, random_state=42, stratify=y_full
)

print(f"\n✅ Pretrain split:")
print(f"   Train: {len(X_train_full)} samples")
print(f"   Val:   {len(X_val_full)} samples")

In [ ]:
# ==========================================
# 10. BUILD PRETRAINING MODEL
# ==========================================

# Build LSTM model for pretraining (2,000 classes)
def build_pretrain_model(input_shape, num_classes):
    """Build LSTM model for pretraining."""
    model = Sequential([
        LSTM(128, return_sequences=True, activation='relu', input_shape=input_shape),
        Dropout(0.2),
        
        LSTM(256, return_sequences=True, activation='relu'),
        Dropout(0.2),
        
        LSTM(128, return_sequences=False, activation='relu'),
        Dropout(0.2),
        
        Dense(128, activation='relu'),
        Dropout(0.3),
        
        Dense(num_classes, activation='softmax')
    ])
    
    return model

# Build pretraining model
INPUT_SHAPE = (TARGET_FRAMES, TOTAL_LANDMARKS * 3)  # (30, 1629)
NUM_CLASSES_FULL = len(all_word_to_videos)  # ~2,000

pretrain_model = build_pretrain_model(INPUT_SHAPE, NUM_CLASSES_FULL)

# Compile
pretrain_model.compile(
    optimizer='Adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

print("✅ Pretraining model built!")
print(f"\n📊 Model Info:")
print(f"   Input shape: {INPUT_SHAPE}")
print(f"   Output classes: {NUM_CLASSES_FULL}")
print(f"   Total parameters: {pretrain_model.count_params():,}")

In [ ]:
# ==========================================
# 11. PRETRAIN MODEL
# ==========================================

# Callbacks
pretrain_checkpoint = ModelCheckpoint(
    '/kaggle/working/lstm_pretrained_2000.h5',
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5,
    min_lr=1e-7,
    verbose=1
)

early_stop = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True,
    verbose=1
)

# Pretrain
print("🚀 Starting pretraining on 2,000 words...")
print(f"   Epochs: 50")
print(f"   Batch size: 32")
print(f"   Training samples: {len(X_train_full)}")
print(f"\n" + "="*60)

pretrain_history = pretrain_model.fit(
    X_train_full, y_train_full,
    validation_data=(X_val_full, y_val_full),
    epochs=50,
    batch_size=32,
    callbacks=[pretrain_checkpoint, reduce_lr, early_stop],
    verbose=1
)

print(f"\n" + "="*60)
print("✅ Pretraining complete!")

In [ ]:
# ==========================================
# 12. FINE-TUNE ON 100 WORDS
# ==========================================

print("🎯 Fine-tuning on your 100 words...")

# Load pretrained model
pretrained = tf.keras.models.load_model('/kaggle/working/lstm_pretrained_2000.h5')

# Create fine-tuning model (100 classes)
def build_finetune_model(pretrained_model, num_classes):
    """Create fine-tuning model from pretrained weights."""
    
    # Get all layers except the final Dense layer
    base_layers = pretrained_model.layers[:-1]
    
    # Create new model
    inputs = pretrained_model.input
    x = inputs
    
    # Add base layers (frozen initially)
    for layer in base_layers:
        x = layer(x)
        layer.trainable = False  # Freeze initially
    
    # Add new classifier
    outputs = Dense(num_classes, activation='softmax')(x)
    
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    
    return model

# Build fine-tuning model
finetune_model = build_finetune_model(pretrained, num_classes)

# Compile with smaller learning rate
finetune_model.compile(
    optimizer=Adam(learning_rate=1e-4),  # Smaller LR for fine-tuning
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

print("✅ Fine-tuning model built!")
print(f"   Base layers frozen: {sum(not layer.trainable for layer in finetune_model.layers)}")
print(f"   Trainable layers: {sum(layer.trainable for layer in finetune_model.layers)}")
print(f"   Total parameters: {finetune_model.count_params():,}")

# Load your 100-word data (reuse from earlier)
print("\n📦 Loading 100-word dataset...")
# This will use the X_data, y_data from earlier cells
# For now, let's assume we have them loaded

# Train/val split for 100 words
X_train_100, X_val_100, y_train_100, y_val_100 = train_test_split(
    X_data, y_data, test_size=0.2, random_state=42, stratify=y_data
)

print(f"   Train: {len(X_train_100)} samples")
print(f"   Val:   {len(X_val_100)} samples")

In [ ]:
# ==========================================
# 13. TRAIN FINE-TUNING MODEL
# ==========================================

# Callbacks for fine-tuning
finetune_checkpoint = ModelCheckpoint(
    '/kaggle/working/lstm_finetuned_100.h5',
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)

# Fine-tune
print("🎯 Starting fine-tuning on 100 words...")
print(f"   Epochs: 30")
print(f"   Batch size: 32")

finetune_history = finetune_model.fit(
    X_train_100, y_train_100,
    validation_data=(X_val_100, y_val_100),
    epochs=30,
    batch_size=32,
    callbacks=[finetune_checkpoint],
    verbose=1
)

print(f"\n✅ Fine-tuning complete!")

In [ ]:
# ==========================================
# 14. EVALUATE & SAVE FINAL MODEL
# ==========================================

# Load best fine-tuned model
best_model = tf.keras.models.load_model('/kaggle/working/lstm_finetuned_100.h5')

# Evaluate on 100-word validation set
print("📊 Evaluating final model on 100 words...")
val_loss, val_acc = best_model.evaluate(X_val_100, y_val_100, verbose=0)

print(f"\n✅ Final Results:")
print(f"   Val Loss:     {val_loss:.4f}")
print(f"   Val Accuracy: {val_acc*100:.2f}%")

# Save final model and labels
best_model.save('/kaggle/working/lstm_wlasl100_final.h5')
print("   ✅ Saved: lstm_wlasl100_final.h5")

# Save label mapping
final_mapping = {
    'gloss_to_label': gloss_to_label,
    'label_to_gloss': label_to_gloss,
    'num_classes': num_classes,
    'model_info': {
        'input_shape': list(INPUT_SHAPE),
        'target_frames': TARGET_FRAMES,
        'total_landmarks': TOTAL_LANDMARKS,
        'val_accuracy': float(val_acc),
        'val_loss': float(val_loss),
        'training_approach': 'pretrain_2000_finetune_100'
    }
}

with open('/kaggle/working/lstm_wlasl100_labels.json', 'w') as f:
    json.dump(final_mapping, f, indent=2)
print("   ✅ Saved: lstm_wlasl100_labels.json")

print(f"\n📥 Download these files for local webcam inference:")
print(f"   1. lstm_wlasl100_final.h5")
print(f"   2. lstm_wlasl100_labels.json")

In [ ]:
# ==========================================
# 8. BUILD LSTM MODEL
# ==========================================

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

# Model architecture (based on Nicholas Renotte's approach)
def build_lstm_model(input_shape, num_classes):
    """Build LSTM model for sign language recognition."""
    model = Sequential([
        # Input: (30, 1629)
        LSTM(64, return_sequences=True, activation='relu', input_shape=input_shape),
        Dropout(0.2),
        
        LSTM(128, return_sequences=True, activation='relu'),
        Dropout(0.2),
        
        LSTM(64, return_sequences=False, activation='relu'),
        Dropout(0.2),
        
        Dense(64, activation='relu'),
        Dropout(0.3),
        
        Dense(num_classes, activation='softmax')
    ])
    
    return model

# Build model
INPUT_SHAPE = (TARGET_FRAMES, TOTAL_LANDMARKS * 3)  # (30, 1629)
NUM_CLASSES = num_classes  # 100

model = build_lstm_model(INPUT_SHAPE, NUM_CLASSES)

# Compile
model.compile(
    optimizer='Adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

print("✅ LSTM Model built!")
print(f"\n📊 Model Summary:")
model.summary()

print(f"\n📊 Model Info:")
print(f"   Input shape: {INPUT_SHAPE}")
print(f"   Output classes: {NUM_CLASSES}")
print(f"   Total parameters: {model.count_params():,}")

In [ ]:
# ==========================================
# 3. MATCH PARQUET FILES TO WORDS
# ==========================================

# Get all parquet files
WLASL_DIR = "/kaggle/input/wlasl2000-landmarks/wlasl_landmarks"
parquet_files = sorted(glob.glob(f"{WLASL_DIR}/*.parquet"))

print(f"? Total parquet files: {len(parquet_files)}")

# We need to find which parquet files correspond to our 100 words
# The WLASL dataset structure: each word has multiple video instances
# We'll need to check if there's a mapping file or use the WLASL metadata

# For now, let's check the nslt_2000.json structure more carefully
WLASL_METADATA = "/kaggle/input/wlasl-processed/nslt_2000.json"

with open(WLASL_METADATA, 'r') as f:
    wlasl_meta = json.load(f)

print(f"\n🔍 Checking WLASL metadata structure...")
print(f"   Type: {type(wlasl_meta)}")

# If it's a dict with video_id as keys
if isinstance(wlasl_meta, dict):
    # Get first entry
    first_key = list(wlasl_meta.keys())[0]
    first_value = wlasl_meta[first_key]
    print(f"\n   Sample entry:")
    print(f"   Key: {first_key}")
    print(f"   Value: {first_value}")
    
    # Check if 'action' field contains the word
    if isinstance(first_value, dict) and 'action' in first_value:
        print(f"\n✅ Found 'action' field! This contains the word labels.")
        
        # Create video_id → word mapping
        video_to_word = {}
        word_to_videos = {}
        
        for video_id, data in wlasl_meta.items():
            word = data.get('action', None)
            if word and word in gloss_to_label:  # Only keep our 100 words
                video_to_word[video_id] = word
                if word not in word_to_videos:
                    word_to_videos[word] = []
                word_to_videos[word].append(video_id)
        
        print(f"\n✅ Mapping created!")
        print(f"   Videos matched to our 100 words: {len(video_to_word)}")
        print(f"   Words found: {len(word_to_videos)}/{num_classes}")
        
        # Show distribution
        print(f"\n📊 Videos per word (sorted by count):")
        sorted_words = sorted(word_to_videos.items(), key=lambda x: len(x[1]), reverse=True)
        for i, (word, videos) in enumerate(sorted_words[:20], 1):
            label = gloss_to_label[word]
            print(f"   {i:2d}. {word:20s} (Label {label:2d}) → {len(videos):3d} videos")
        
        # Check coverage
        missing_words = set(gloss_to_label.keys()) - set(word_to_videos.keys())
        if missing_words:
            print(f"\n⚠️ Missing {len(missing_words)} words from our 100-word list:")
            print(f"   {sorted(list(missing_words))[:10]}...")
        else:
            print(f"\n✅ All 100 words have video data!")

print(f"\n" + "="*60)
print(f"✅ DATA EXPLORATION COMPLETE!")
print(f"="*60)